In [1]:
%pylab inline

import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


# Готовим данные

объединяем таблицы за март-май

In [2]:
data = pd.read_csv('tabl_05_2016.csv', sep = ';')
data['mean'] = [sum(data.loc[i])/data.shape[1] for i in range(data.shape[0])]
data = data.loc[data['mean'] > 5]
data.drop(['mean'], axis='columns', inplace = True)
data.head()

,2016.05.01 00:00,2016.05.01 01:00,2016.05.01 02:00,2016.05.01 03:00,2016.05.01 04:00,2016.05.01 05:00,2016.05.01 06:00,2016.05.01 07:00,2016.05.01 08:00,2016.05.01 09:00,...,2016.05.31 14:00,2016.05.31 15:00,2016.05.31 16:00,2016.05.31 17:00,2016.05.31 18:00,2016.05.31 19:00,2016.05.31 20:00,2016.05.31 21:00,2016.05.31 22:00,2016.05.31 23:00
1075,71.0,37.0,10.0,14.0,6.0,8.0,10.0,21.0,17.0,23.0,...,124.0,105.0,80.0,84.0,106.0,106.0,99.0,84.0,83.0,34.0
1076,64.0,39.0,46.0,25.0,20.0,12.0,16.0,48.0,61.0,83.0,...,214.0,208.0,156.0,202.0,213.0,167.0,166.0,173.0,98.0,77.0
1077,52.0,13.0,8.0,6.0,4.0,6.0,7.0,16.0,24.0,64.0,...,104.0,69.0,67.0,109.0,173.0,171.0,165.0,107.0,127.0,58.0
1125,81.0,38.0,34.0,23.0,22.0,5.0,23.0,18.0,15.0,27.0,...,96.0,70.0,68.0,109.0,126.0,88.0,65.0,78.0,94.0,53.0
1126,259.0,121.0,86.0,59.0,64.0,46.0,68.0,69.0,89.0,112.0,...,205.0,213.0,168.0,250.0,281.0,293.0,301.0,256.0,174.0,118.0


In [3]:
data_4 = pd.read_csv('tabl_04_2016.csv', sep = ';')
data_4 = data_4.loc[data.index]
data = data_4.join(data, how = 'outer')
data.head()

,2016.04.01 00:00,2016.04.01 01:00,2016.04.01 02:00,2016.04.01 03:00,2016.04.01 04:00,2016.04.01 05:00,2016.04.01 06:00,2016.04.01 07:00,2016.04.01 08:00,2016.04.01 09:00,...,2016.05.31 14:00,2016.05.31 15:00,2016.05.31 16:00,2016.05.31 17:00,2016.05.31 18:00,2016.05.31 19:00,2016.05.31 20:00,2016.05.31 21:00,2016.05.31 22:00,2016.05.31 23:00
1075,46.0,25.0,12.0,8.0,9.0,7.0,16.0,43.0,70.0,59.0,...,124.0,105.0,80.0,84.0,106.0,106.0,99.0,84.0,83.0,34.0
1076,55.0,44.0,23.0,9.0,13.0,28.0,76.0,154.0,217.0,205.0,...,214.0,208.0,156.0,202.0,213.0,167.0,166.0,173.0,98.0,77.0
1077,31.0,16.0,13.0,4.0,4.0,13.0,48.0,146.0,200.0,145.0,...,104.0,69.0,67.0,109.0,173.0,171.0,165.0,107.0,127.0,58.0
1125,67.0,46.0,20.0,20.0,16.0,19.0,42.0,77.0,103.0,81.0,...,96.0,70.0,68.0,109.0,126.0,88.0,65.0,78.0,94.0,53.0
1126,171.0,90.0,58.0,32.0,33.0,63.0,115.0,150.0,237.0,224.0,...,205.0,213.0,168.0,250.0,281.0,293.0,301.0,256.0,174.0,118.0


In [4]:
data_3 = pd.read_csv('tabl_03_2016.csv', sep = ';')
data_3 = data_3.loc[data.index]
data = data_3.join(data, how = 'outer')
data.head()

,2016.03.01 00:00,2016.03.01 01:00,2016.03.01 02:00,2016.03.01 03:00,2016.03.01 04:00,2016.03.01 05:00,2016.03.01 06:00,2016.03.01 07:00,2016.03.01 08:00,2016.03.01 09:00,...,2016.05.31 14:00,2016.05.31 15:00,2016.05.31 16:00,2016.05.31 17:00,2016.05.31 18:00,2016.05.31 19:00,2016.05.31 20:00,2016.05.31 21:00,2016.05.31 22:00,2016.05.31 23:00
1075,14.0,16.0,4.0,2.0,5.0,9.0,20.0,30.0,80.0,63.0,...,124.0,105.0,80.0,84.0,106.0,106.0,99.0,84.0,83.0,34.0
1076,22.0,15.0,14.0,7.0,16.0,23.0,62.0,168.0,211.0,200.0,...,214.0,208.0,156.0,202.0,213.0,167.0,166.0,173.0,98.0,77.0
1077,20.0,12.0,0.0,1.0,2.0,13.0,69.0,168.0,212.0,144.0,...,104.0,69.0,67.0,109.0,173.0,171.0,165.0,107.0,127.0,58.0
1125,39.0,12.0,9.0,4.0,8.0,9.0,47.0,81.0,104.0,88.0,...,96.0,70.0,68.0,109.0,126.0,88.0,65.0,78.0,94.0,53.0
1126,78.0,58.0,23.0,16.0,30.0,49.0,104.0,139.0,212.0,198.0,...,205.0,213.0,168.0,250.0,281.0,293.0,301.0,256.0,174.0,118.0


In [5]:
data_6 = pd.read_csv('tabl_06_2016.csv', sep = ';')
data_6 = data_6.loc[data.index]
data = data.join(data_6, how = 'outer')
data.head()

,2016.03.01 00:00,2016.03.01 01:00,2016.03.01 02:00,2016.03.01 03:00,2016.03.01 04:00,2016.03.01 05:00,2016.03.01 06:00,2016.03.01 07:00,2016.03.01 08:00,2016.03.01 09:00,...,2016.06.30 14:00,2016.06.30 15:00,2016.06.30 16:00,2016.06.30 17:00,2016.06.30 18:00,2016.06.30 19:00,2016.06.30 20:00,2016.06.30 21:00,2016.06.30 22:00,2016.06.30 23:00
1075,14.0,16.0,4.0,2.0,5.0,9.0,20.0,30.0,80.0,63.0,...,109.0,86.0,81.0,82.0,88.0,116.0,104.0,151.0,106.0,85.0
1076,22.0,15.0,14.0,7.0,16.0,23.0,62.0,168.0,211.0,200.0,...,236.0,199.0,152.0,139.0,173.0,190.0,142.0,162.0,168.0,130.0
1077,20.0,12.0,0.0,1.0,2.0,13.0,69.0,168.0,212.0,144.0,...,103.0,68.0,85.0,100.0,117.0,135.0,149.0,145.0,103.0,86.0
1125,39.0,12.0,9.0,4.0,8.0,9.0,47.0,81.0,104.0,88.0,...,107.0,78.0,86.0,80.0,75.0,132.0,141.0,135.0,125.0,113.0
1126,78.0,58.0,23.0,16.0,30.0,49.0,104.0,139.0,212.0,198.0,...,212.0,213.0,179.0,255.0,309.0,395.0,333.0,359.0,317.0,256.0


In [6]:
index = data.index

In [7]:
K = 23
def pred(data, K):
    year_period = 8766 #365.2424*24 - средняя продолжительность года в часах
    week_period = 168
    day_period = 24
    for i in range(1, K+1):
        data['y_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
        data['y_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
    for i in range(1, K+1):    
        data['w_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
        data['w_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
    for i in range(1, K+1):    
        data['d_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)
        data['d_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)

# Добавляем признаки

In [8]:
def add(i):
    data_copy = pd.DataFrame(data.loc[i])
    data_copy.columns = ['trip'] #количество поездок
    data_copy['region'] = [str(i)]*data.shape[1] #регион
    hours = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').hour for i in data.columns] 
    data_copy['hour'] = hours                                                          #час
    month = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').month for i in data.columns]
    data_copy['month'] = month                                                        #месяц
    days = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').day for i in data.columns] 
    data_copy['days'] = days                            #число
    year = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').year for i in data.columns] 
    data_copy['year'] = year  
    dayofweek = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').weekday() for i in data.columns]
    data_copy['dayofweek'] = dayofweek                                               #день недели
    
    weekends = []

    
    for j in range(len(data_copy)):
        if data_copy['dayofweek'][j] == '1' or data_copy['dayofweek'][j] == '7':
            weekends.append(1)
        else:
            weekends.append(0)

    data_copy['weekend'] = weekends # является ли день выходным
    data_copy['y-1'] = data_copy.trip.shift(1) #количество поездок за предыдущие n часов (1, 2, 3, 4, 5, 6, 24, 48)
    data_copy['y-2'] = data_copy.trip.shift(2)
    data_copy['y-3'] = data_copy.trip.shift(3)
    data_copy['y-7'] = data_copy.trip.shift(7)
    data_copy['y-8'] = data_copy.trip.shift(8)
    data_copy['y-9'] = data_copy.trip.shift(9)
    
    data_copy['y-24'] = data_copy.trip.shift(24)
    data_copy['y-48'] = data_copy.trip.shift(48)
    
    pred(data_copy, K)
    return data_copy.dropna()

# Настраиваем модель для каждого региона

In [11]:
%%time
models = {}
y_dict = {}
warnings.filterwarnings('ignore')

for i in index:
    data_copy = add(i)
    print(i, end = ' ')
    train_stop = '2016.05.01 00:00'
    test_stop = '2016.06.01 00:00'
    X_train = data_copy[data_copy.columns[1:]].loc[:train_stop]
    y_train = data_copy[data_copy.columns[:1]].loc[:train_stop]
    
    X_test = data_copy[data_copy.columns[1:]].loc[train_stop: test_stop]
    y_test = data_copy[data_copy.columns[:1]].loc[train_stop: test_stop]
    
    X_pred = data_copy[data_copy.columns[1:]].loc[test_stop:]
    y_pred = data_copy[data_copy.columns[:1]].loc[test_stop:]
    
    mse_min = mean_squared_error(y_test, [0]*len(y_test))
    for alpha in np.logspace(-2, 3, 100):
        model = linear_model.Lasso(alpha = alpha)
        model.fit(X_train, y_train)
        mse = mean_squared_error(model.predict(X_test), y_test)
        if mse < mse_min:
            best_model = model
            mse_min = mse
        
    models[i] = best_model
    y = best_model.predict(X_pred)
    y_dict[i] = y

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 Wall time: 15min 59s


# Считаем ошибку, создаем файл для соревнования

In [12]:
s = 0
ys = []
for i in index:
    for j in range(data_6.shape[1]-5):
        for k in range(0, 6):
            s += abs(y_dict[i][j+k] - data_6.loc[i][j+k])
            #ind = str(i) + '_2016-' + t[3:5] + '-' + t[0:2] + '_' + str(int(t[6:8])) + '_' + str(i+1)
            ys.append(y_dict[i][j+k])
            
    print('i = ', i, 's = ', s)

i =  1075 s =  54794.749765504195
i =  1076 s =  130112.43213604332
i =  1077 s =  188554.77021048908
i =  1125 s =  230336.50669078712
i =  1126 s =  336887.9910704378
i =  1127 s =  485384.45354459924
i =  1128 s =  640750.0782243891
i =  1129 s =  822897.7510005995
i =  1130 s =  1035892.5821487746
i =  1131 s =  1147231.8010174509
i =  1132 s =  1211950.310061065
i =  1172 s =  1221680.2645890568
i =  1173 s =  1248215.6964925874
i =  1174 s =  1276119.2415338764
i =  1175 s =  1293053.5914140746
i =  1176 s =  1312837.0023174095
i =  1177 s =  1518902.1558628345
i =  1178 s =  1780621.2765785404
i =  1179 s =  2021906.1108229484
i =  1180 s =  2296758.2482054317
i =  1181 s =  2701806.3867450966
i =  1182 s =  2923990.9666455826
i =  1183 s =  3030715.008097321
i =  1184 s =  3049144.262611763
i =  1221 s =  3060221.6396666924
i =  1222 s =  3073757.38558222
i =  1223 s =  3093147.7492643893
i =  1224 s =  3116374.0828887415
i =  1225 s =  3126354.6011395524
i =  1227 s =  3252450

In [13]:
a = pd.read_csv('a.csv', sep = ',')
a['y'] = ys

In [14]:
a.to_csv('a_3.csv', sep = ',', index = False)

In [15]:
s/739/102/6

20.453199121616912

<img src = '2020-12-08_23-53-58.png'>